## < DRAFT > Methods for Processing NFHP Habitat Condition and Disturbances to  Ecological and Jurisdictional  Units of Interest 
#### Daniel Wieferich -USGS

This code summarizes NFHP Fish Habitat Condition Indicies to areas of ecological and jurisdictional units of interest (e.g. DOI lands, States, Ecoregions...), allowing us to understand fish habitat condition in units of management concern.  To create summaries first midpoints of each NFHP scored NHDPlusV1 flowline are assigned to spatial units that they fall within.  Habitat condition scores are summarized using a length-weighted average approach and disturbance variables for each of the four spatial scales are summarized by combining lists of disturbances from individual stream reaches into a common list of unique values.  These summaries are intended to be used in the National Biogeographic Map.

This code is in progress and may change through time.

#### Summaries to calculate per spatial unit
    #1.length-weighted average hci per spatial unit
    #2.total stream kilometers that were scored by NFHP within the spatial unit 
    #3.total stream kilometers that were not scored by NFHP within the spatial unit 
    #4.list of all significant anthropogenic disturbances effecting habitat condition within the spatial unit
    #5.list of disturbances spatial unit for each spatial unit (lb, lc, nb, nc)
    #6.stream kilometers within each disturbance class (e.g. very low risk to very high risk) within the spatial unit
    
    Below summaries still need to be addressed
    #7.most pervasive disturbance effecting habitat condition within the spatial unit
    #8.most severe disturbance effecting habitat condition within the spatial unit
    

In [ ]:
#Import needed packages
import requests
import pandas as pd
from bis2 import gc2
import warnings
import geojson
import geopandas as gpd
import numpy as np

warnings.filterwarnings("ignore")



In [ ]:
#Creates table in database to insert resulting summaries 

q_createhci = "CREATE TABLE IF NOT EXISTS nfhp.hci2015_summaries_mp ( \
    source_id varchar(250) primary key,\
    place_name varchar(250),\
    lc_hci2015_len_weight double precision,\
    nc_hci2015_len_weight double precision,\
    lb_hci2015_len_weight double precision,\
    nb_hci2015_len_weight double precision,\
    cumu_hci2015_len_weight double precision,\
    nc_list_dist text[],\
    lc_list_dist text[],\
    lb_list_dist text[],\
    nb_list_dist text[],\
    cumu_list_dist text[],\
    scored_km double precision,\
    not_scored_km double precision,\
    verylow_km double precision,\
    low_km double precision,\
    moderate_km double precision,\
    high_km double precision,\
    veryhigh_km double precision)"
url_createhci = gc2.sqlAPI("datadistillery","bcb_beta")+"&q="+q_createhci
print (requests.get(url_createhci,verify=False).json())


In [ ]:
#Defines Database requirements

thisRun = {}
thisRun['url'] = gc2.baseURLs["sqlapi_datadistillery_bcb_beta"]
thisRun['gc2Key'] = gc2.gc2Keys['datadistillery_bcb']  #Reads in api key, allowing for insert capabilities

thisRun['fromSchema'] = 'sfr.'
thisRun['fromTable'] = 'placenamelookup_poly'


thisRun['insertSchema'] = 'nfhp.'
thisRun['insertTable'] = 'hci2015_summaries_mp'
thisRun['insertId'] = 'source_id'

thisRun['fromSchemaTable'] = thisRun['fromSchema']+thisRun['fromTable']
thisRun['insertSchemaTable'] = thisRun['insertSchema']+thisRun['insertTable']

In [ ]:
#create csv to store disturbance variable information.  This is a temporary solution that is still being looked into.
# This csv file currently reaches over 5GB which GC2 can not handle for upload.  Inserting row by row is SLOW so either the
# data need to be processed concurrently with the length-weighted processes in this code or another data solution needs to be 
#explored. 


import csv
fieldNames = 'source_id, comid, spatial_scale, disturbance  \n'
outFile = open('limiting_2015_mp_l.csv','a')
outFile.write(fieldNames)
outFile.close()

In [ ]:
#===============================================================================================================
#===============================================================================================================
# toProcessValidation identifies features that still need to be processed by comparing features in a from table
# to the feature ids in the insert table (aka to table)
#===============================================================================================================
def toProcessValidation (thisRun):
    #Retrieves a list of spatial units that need to be processed
    toRun = []
    ident = thisRun['insertId']
    
    queryUnits = thisRun['url']+"?q=select " + thisRun['fromSchemaTable'] + "." + ident + " from " +\
    thisRun['fromSchemaTable'] + " left join " + thisRun['insertSchemaTable'] + " on " +  \
    thisRun['insertSchemaTable'] + "." + ident + " = " + thisRun['fromSchemaTable']+ "." + ident +\
    " where " + thisRun['insertSchemaTable'] + "." + ident + " is null"
    
    featuresJson = requests.get(url=queryUnits,verify=False).json()
    
    if featuresJson != []:
        toRunFeatures = featuresJson['features']
        for feature in toRunFeatures:
            toRun.append(feature['properties'][ident])
    
    return toRun

#===============================================================================================================
#===============================================================================================================
#===============================================================================================================

def requestHci(url, place):
    import requests
    import geopandas as gpd
    import geojson

    
    queryHci = url+"?q=select place_flow.comid, hci.lc_hci, hci.nc_hci, hci.lb_hci, hci.nb_hci, hci.cumu_hci, hci.lc_limit, hci.lb_limit, hci.nc_limit, hci.nb_limit, hci.cu_hcitext, place_flow.place_name, place_flow.lengthkm as seg_length from \
(select flow.comid, flow.lengthkm, place.place_name from nhd.nhdplusv1_midpoint_5070 flow inner join \
(select place_name, the_geom from sfr.placenamelookup_poly where source_id='" + place + "') place on ST_Within(flow.the_geom,place.the_geom) where flow.ftype in ('ArtificialPath','StreamRiver','Connector','CanalDitch')) place_flow \
left join nfhp.nfhp2015_hci hci on hci.comid=place_flow.comid"
    
#    print (queryHci)
    hciRequest = requests.get(url=queryHci, verify=False)
    hciData = geojson.loads(hciRequest.text)
    hciDf = gpd.GeoDataFrame.from_features(hciData['features'])
    hciDf.crs ={'init':'epsg:5070'}
    
    return hciDf

#===============================================================================================================
#===============================================================================================================
# hciSummaries calculates length weighted summaries of nfhp hci scores per feature polygon,
# calculates lists of disturbances within the feature polygon and preps disturbance data to allow for 
# designations of pervasive and severe disturbances within the feature polygon (distValue dictionary)
#===============================================================================================================

def hciSummaries(hciDf):

    #Summaries to calculate, followed by comment of definition
    nc_limit = [] #unique list of upstream network catchment limiting disturbances
    nb_limit = [] #unique list of upstream network buffer limiting disturbances
    lc_limit = [] #unique list of local catchment limiting disturbances
    lb_limit = [] #unique list of local buffer limiting disturbances
    distValue = [] #list of dictionaries to populate disturbance per feature polygon table
    notScored = 0  #Number of stream km not scored by NFHP within the feature polygon
    scored = 0     #Number of stream km scored by NFHP within the feature polygon
    weightCumu = 0 #length-weighted average summary of cumulative hci score
    weightLc = 0   #length-weighted average summary of local catchment hci score
    weightLb = 0   #length-weighted average summary of local buffer hci score
    weightNc = 0   #length-weighted average summary of network catchment hci score
    weightNb = 0   #length-weighted average summary of network buffer hci score
    veryLow = 0
    low = 0
    mod = 0
    high = 0
    veryHigh = 0

    
    for row in hciDf.itertuples():


        #Calculates total stream km with and without habitat condition score within the feature polygon
        #only run on rows that have habitat condition index values, this allows us to capture scored vs not scored kms

        if row.cumu_hci:

            if row.cu_hcitext:
                if row.cu_hcitext == 'Very low':
                    veryLow += float(row.seg_length)
                elif row.cu_hcitext == 'Low':
                    low += float(row.seg_length)
                elif row.cu_hcitext == 'Moderate':
                    mod += float(row.seg_length)
                elif row.cu_hcitext == 'High':
                    high += float(row.seg_length)
                elif row.cu_hcitext == 'Very high':
                    veryHigh += float(row.seg_length)

                        
            scored += float(row.seg_length)
            if row.cumu_hci:
                weightCumu += (float(row.seg_length))*(float(row.cumu_hci))
            if row.lc_hci:
                weightLc += (float(row.seg_length))*(float(row.lc_hci))
            if row.lb_hci:
                weightLb += (float(row.seg_length))*(float(row.lb_hci))
            if row.nc_hci:
                weightNc += (float(row.seg_length))*(float(row.nc_hci))
            if row.nb_hci:
                weightNb += (float(row.seg_length))*(float(row.nb_hci))
            
            #Calculating List Disturbances, Most Pervasive and Most Severe
            if row.nc_limit:
                for word in row.nc_limit.split(';'):
                    nc_limit.append(word)
                    distValue.append({'comid':int(row.comid), 'spatial_scale':'nc', 'disturbance':word})
            if row.nb_limit:
                for word in row.nb_limit.split(';'):
                    nb_limit.append(word)
                    distValue.append({'comid':int(row.comid), 'spatial_scale':'nb', 'disturbance':word})
            if row.lc_limit:
                for word in row.lc_limit.split(';'):
                    lc_limit.append(word)
                    distValue.append({'comid':int(row.comid), 'spatial_scale':'lc', 'disturbance':word})
            if row.lb_limit:
                for word in row.lb_limit.split(';'):
                    lb_limit.append(word)
                    distValue.append({'comid':int(row.comid), 'spatial_scale':'lb', 'disturbance':word})
        
        #for unscored streams track cumulative unscored stream km
        else:
            notScored += float(row.seg_length)
            

    summaryValues = {'nc_list_dist':list(set(nc_limit)),'nb_list_dist':list(set(nb_limit)),
                     'lc_list_dist':list(set(lc_limit)),'lb_list_dist':list(set(lb_limit)),
                     'cumu_list_dist':list(set(lb_limit+lc_limit+nb_limit+nc_limit)),
                    'lc_hci2015_len_weight':weightLc/scored, 'nc_hci2015_len_weight': weightNc/scored,
                    'lb_hci2015_len_weight':weightLb/scored, 'nb_hci2015_len_weight': weightNb/scored,
                    'cumu_hci2015_len_weight':weightCumu/scored, 
                    'not_scored_km':notScored, 'scored_km':scored, 'verylow_km': veryLow, 'low_km': low, 'moderate_km': mod,
                    'high_km': high, 'veryhigh_km': veryHigh}
    
    return distValue, summaryValues

In [ ]:
#Identify and open csv file to dump disturbance data into
outFileName = 'limiting_2015_mp_l.csv'
outFile = open(outFileName,'a')


#Capture spatial units where code fails
failLog = []

#Determines which spatial units still need to be summarized
toRun = toProcessValidation (thisRun)
while toRun != []:
    
    for run in toRun:
        try:
            #Run function to request data for a given spatial unit
            hciDf = requestHci(thisRun['url'], run)
            # Summarize data for a given spatial unit, return results
            distValue, summaryValues = hciSummaries(hciDf)


            #Query for inserting resulting summaries into BIS Database 
            summaryInsert = "insert into nfhp.hci2015_summaries_mp(source_id, place_name, lc_hci2015_len_weight, nc_hci2015_len_weight, \
lb_hci2015_len_weight, nb_hci2015_len_weight, cumu_hci2015_len_weight, nc_list_dist, lc_list_dist, lb_list_dist, \
nb_list_dist, cumu_list_dist, scored_km, not_scored_km, verylow_km, low_km, moderate_km, high_km, veryhigh_km) VALUES ('" + str(run) + "' ,'"  + str(hciDf['place_name'][0]) + "' ,'" \
+ str(summaryValues['lc_hci2015_len_weight']) + "' ,'" + str(summaryValues['nc_hci2015_len_weight']) + "' ,'" + \
str(summaryValues['lb_hci2015_len_weight'])+ "' ,'" + str(summaryValues['nb_hci2015_len_weight'])+ "' ,'" + \
str(summaryValues['cumu_hci2015_len_weight'])+ "' , ARRAY" + str(summaryValues['nc_list_dist'])+ "  , ARRAY" + \
str(summaryValues['lc_list_dist'])+ ", ARRAY" + str(summaryValues['lb_list_dist'])+ " , ARRAY" + \
str(summaryValues['nb_list_dist']) + ", ARRAY" + str(summaryValues['cumu_list_dist'])+ ",'" \
+ str(summaryValues['scored_km'])+ "' ,'" + str(summaryValues['not_scored_km'])+ "' ,'" + str(summaryValues['verylow_km']) \
+ "' ,'" + str(summaryValues['low_km'])+ "' ,'" + str(summaryValues['moderate_km']) \
+ "' ,'" + str(summaryValues['high_km'])+ "' ,'" + str(summaryValues['veryhigh_km'])+"')"
    
            #Code that inserts data
            payload = "?q=%s&key=%s"%(summaryInsert,thisRun['gc2Key'])
            queryUrl = thisRun['url']+payload
            r = requests.get(queryUrl, verify=False)
            
            
            #write rows of disturbance values into the csv (the thought was to add the csv to BIS database through UI upon completion)
            for record in distValue:
                newRow = (str(run) + ", " + str(record['comid']) + ", " + str(record['spatial_scale']) + " ," +str(record['disturbance'])+ '\n')
                outFile.write(newRow)
            
    
            
        except:
            #print (run + ' did not work')
            failLog.append(str(run))
            continue
            
outFile.close()

In [ ]:
#The code below runs summaries on one spatial feature (e.g. lcc, state, ecoregion...) from the placenamelookup table. 
#This code was used for testing and also times the process.

import time
run = 'gu_state:e57dfa3f-244d-4dbe-9d45-c034dbda861a'
start_time = time.clock()
try:
    hciDf = requestHci(thisRun['url'], run)
    time1 = time.clock()
    print (str((time1 - start_time)/60) + ' minutes first process')
    distValue, summaryValues = hciSummaries(hciDf)
    time2 = time.clock()
    print (str((time2 - time1)/60) + ' minutes 2nd process')
    
#    result = mongoDist.insert_many(distValue)
    
    
    summaryInsert = "insert into nfhp.hci2015_summaries_mp(source_id, place_name, lc_hci2015_len_weight, nc_hci2015_len_weight, \
lb_hci2015_len_weight, nb_hci2015_len_weight, cumu_hci2015_len_weight, nc_list_dist, lc_list_dist, lb_list_dist, \
nb_list_dist, cumu_list_dist, scored_km, not_scored_km, verylow_km, low_km, moderate_km, high_km, veryhigh_km) VALUES ('" + str(run) + "' ,'"  + str(hciDf['place_name'][0]) + "' ,'" \
+ str(summaryValues['lc_hci2015_len_weight']) + "' ,'" + str(summaryValues['nc_hci2015_len_weight']) + "' ,'" + \
str(summaryValues['lb_hci2015_len_weight'])+ "' ,'" + str(summaryValues['nb_hci2015_len_weight'])+ "' ,'" + \
str(summaryValues['cumu_hci2015_len_weight'])+ "' , ARRAY" + str(summaryValues['nc_list_dist'])+ "  , ARRAY" + \
str(summaryValues['lc_list_dist'])+ ", ARRAY" + str(summaryValues['lb_list_dist'])+ " , ARRAY" + \
str(summaryValues['nb_list_dist']) + ", ARRAY" + str(summaryValues['cumu_list_dist'])+ ",'" \
+ str(summaryValues['scored_km'])+ "' ,'" + str(summaryValues['not_scored_km'])+ "' ,'" + str(summaryValues['verylow_km']) \
+ "' ,'" + str(summaryValues['low_km'])+ "' ,'" + str(summaryValues['moderate_km']) \
+ "' ,'" + str(summaryValues['high_km'])+ "' ,'" + str(summaryValues['veryhigh_km'])+"')"
    
    payload = "?q=%s&key=%s"%(summaryInsert,thisRun['gc2Key'])
    queryUrl = thisRun['url']+payload
    r = requests.get(queryUrl, verify=False)
    
    
    
    outFileName = 'limiting_2015_mp_l.csv'
    outFile = open(outFileName,'a')
    for record in distValue:
        newRow = (str(run) + ", " + str(record['comid']) + ", " + str(record['spatial_scale']) + " ," +str(record['disturbance'])+ '\n')
        #print (newRow)
        outFile.write(newRow)
    outFile.close()
    
except:
    print (run + ' did not work')


end_time = time.clock()
print (str((end_time-start_time)/60) + " minutes total")

In [ ]:
########################BELOW IS TESTING TO SUMMARIZE DISTURBANCE INFORMATON: STILL IN PROGRESS################

In [ ]:
#Group by to summarize cumulative (not specific to scale or stream size) most severe disturbances
listDf = pd.DataFrame(distValue)
listDf.head(25)

In [ ]:
severeAll = listDf.loc[listDf['cu_hcitext'].isin(['Very high','High'])]
severeAll.segLen = severeAll.segLen.apply(float)
groupDf = severeAll.groupby(['comid','disturbance']).mean()
groupDf.reset_index(inplace=True)
sumGroupDf = groupDf.groupby(['disturbance']).sum()
sumGroupDf.reset_index(inplace=True)
del sumGroupDf['comid']
#sumGroupDf

sumGroupDf


In [ ]:
sumGroupDf.groupby(['disturbance'], sort=False)['segLen'].max()

In [ ]:
sumGroupDf